## FLYBALL OPTIMIZATION 
Építsetek egy modellt, amely optimalizálja a csapatösszeállításokat az ismert adatok (kutyák alapadatai, kutyák sorrendje, akadályok magassága, időjárási körülmények) függvényében. 

In [3]:
import pandas as pd
import sasoptpy as so
import os
import time

### ADAT

In [9]:
#beolvassunk 3 táblát, amik a kutyák adatait tartalmazzák:

#kutya_all_features
opt1 = pd.read_csv('./../data/kutya_all_features.csv', index_col = [0])

#flyball_weather
opt2 = pd.read_csv('./../data/flyball_weather.csv', index_col = [0])

#dog_hurdle
opt3 = pd.read_csv('./../data/dog_hurdle.csv', index_col = [0])

#rank_df
opt4 = pd.read_csv('./../data/rank_df.csv', index_col = [0])

In [17]:
#Adatok: mi kell a modellezéshez

opt1.reset_index(inplace=True)
opt1.head()

,index,r_min,r_median,r_mean,r_max,kutya_ev_honapok,Border/Other,min,median,mean,max,futasok_szama,valto_hiba,start_hiba,futas_hiba,összes_hiba,hiba_szazalek
0,Abby,0.00,0.22,0.31,1.25,55,Border,4.32,4.51,4.75,5.75,64.0,8.0,0.0,0.0,8.0,12.50
1,Agnes,0.13,0.56,0.57,1.45,101,Border,4.55,4.73,5.02,7.19,33.0,4.0,0.0,1.0,5.0,15.15
2,Alex,0.00,0.15,0.15,0.55,118,Border,4.02,4.27,4.27,4.56,130.0,5.0,0.0,1.0,6.0,4.62
3,Alfie,0.00,0.13,0.13,0.89,71,Other,3.79,3.98,3.99,4.36,218.0,20.0,0.0,3.0,23.0,10.55
4,Angie,0.00,0.43,0.48,1.55,120,Border,5.07,5.50,5.72,8.36,34.0,3.0,0.0,3.0,6.0,17.65


### Modellépítés

In [16]:
model = so.Model(name='Flyball_model')

NOTE: Initialized model Flyball_model.


In [18]:
#az összes kutya közül válogassunk?
opt1['futasok_szama'].describe()

count     50.000000
mean      89.380000
std       60.949591
min        5.000000
25%       41.250000
50%       71.500000
75%      130.000000
max      227.000000
Name: futasok_szama, dtype: float64

In [20]:
#kutyák listába:
kutyak = opt1.index.tolist()


In [21]:
#Változók bevezetése: a váltó négy helyén futó négy kutya

valto_1 = model.add_variables(kutyak, name='valto_1', vartype=so.binary)

valto_2 = model.add_variables(kutyak, name='valto_2', vartype=so.binary)

valto_3 = model.add_variables(kutyak, name='valto_3', vartype=so.binary)

valto_4 = model.add_variables(kutyak, name='valto_4', vartype=so.binary)

In [22]:
valto_1[3]

sasoptpy.Variable(name='valto_1[3]', lb=0, ub=1, vartype='BIN')

In [24]:
#Object: a váltó négy helyén futó négy kutya medián idejének az összege

total_time = so.expr_sum(valto_1[i] * opt1.loc[i, 'median'] for i in kutyak) \
+ so.expr_sum(valto_2[i] * opt1.loc[i, 'median'] for i in kutyak) \
+ so.expr_sum(valto_3[i] * opt1.loc[i, 'median'] for i in kutyak) \
+ so.expr_sum(valto_4[i] * opt1.loc[i, 'median'] for i in kutyak)

In [25]:
total_time

sasoptpy.Expression(exp = 4.51 * valto_1[0] + 4.73 * valto_1[1] + 4.27 * valto_1[2] + 3.98 * valto_1[3] + 5.5 * valto_1[4] + 4.19 * valto_1[5] + 4.26 * valto_1[6] + 4.66 * valto_1[7] + 4.7 * valto_1[8] + 4.23 * valto_1[9] + 4.21 * valto_1[10] + 4.58 * valto_1[11] + 4.21 * valto_1[12] + 4.78 * valto_1[13] + 4.04 * valto_1[14] + 4.57 * valto_1[15] + 4.29 * valto_1[16] + 6.23 * valto_1[17] + 4.46 * valto_1[18] + 3.96 * valto_1[19] + 4.15 * valto_1[20] + 4.4 * valto_1[21] + 4.51 * valto_1[22] + 6.92 * valto_1[23] + 4.57 * valto_1[24] + 4.23 * valto_1[25] + 4.76 * valto_1[26] + 5.85 * valto_1[27] + 5.01 * valto_1[28] + 8.68 * valto_1[29] + 4.93 * valto_1[30] + 4.03 * valto_1[31] + 4.72 * valto_1[32] + 4.3 * valto_1[33] + 3.96 * valto_1[34] + 4.95 * valto_1[35] + 4.14 * valto_1[36] + 4.09 * valto_1[37] + 5.91 * valto_1[38] + 5.04 * valto_1[39] + 4.5 * valto_1[40] + 5.18 * valto_1[41] + 4.55 * valto_1[42] + 4.77 * valto_1[43] + 4.06 * valto_1[44] + 5.11 * valto_1[45] + 4.07 * valto_1[46] + 4.

In [26]:
#Cél - ez az összeg legyen minimális
model.set_objective(total_time, name='total_time_obj', sense=so.minimize)

sasoptpy.Expression(exp = 4.51 * valto_1[0] + 4.73 * valto_1[1] + 4.27 * valto_1[2] + 3.98 * valto_1[3] + 5.5 * valto_1[4] + 4.19 * valto_1[5] + 4.26 * valto_1[6] + 4.66 * valto_1[7] + 4.7 * valto_1[8] + 4.23 * valto_1[9] + 4.21 * valto_1[10] + 4.58 * valto_1[11] + 4.21 * valto_1[12] + 4.78 * valto_1[13] + 4.04 * valto_1[14] + 4.57 * valto_1[15] + 4.29 * valto_1[16] + 6.23 * valto_1[17] + 4.46 * valto_1[18] + 3.96 * valto_1[19] + 4.15 * valto_1[20] + 4.4 * valto_1[21] + 4.51 * valto_1[22] + 6.92 * valto_1[23] + 4.57 * valto_1[24] + 4.23 * valto_1[25] + 4.76 * valto_1[26] + 5.85 * valto_1[27] + 5.01 * valto_1[28] + 8.68 * valto_1[29] + 4.93 * valto_1[30] + 4.03 * valto_1[31] + 4.72 * valto_1[32] + 4.3 * valto_1[33] + 3.96 * valto_1[34] + 4.95 * valto_1[35] + 4.14 * valto_1[36] + 4.09 * valto_1[37] + 5.91 * valto_1[38] + 5.04 * valto_1[39] + 4.5 * valto_1[40] + 5.18 * valto_1[41] + 4.55 * valto_1[42] + 4.77 * valto_1[43] + 4.06 * valto_1[44] + 5.11 * valto_1[45] + 4.07 * valto_1[46] + 4.

In [30]:
#Korlátok bevezetése

In [30]:
#a váltó tagjainak különbözniük kell:

model.add_constraints((valto_1[i] + valto_2[i] + valto_3[i] + valto_4[i] <= 1 for i in kutyak), name='diff_valto');

In [33]:
#egy kutya lehet egyidőben a váltó 1 pontján:

model.add_constraints(so.expr_sum(valto_1[i] for i in kutyak) == 1, name ='first_kutya');
#ugyanezt a váltó többi három helyére is...

#Többi korlátot megfogalmazni:
- gát magassága
- poziciókban futó kutyák
- időjárási tényezők